## Lyrics analysis and comparison of three Italian singers

#### Computational skills for text analysis 2020/21 - Final project

STEP 1 - Scraping and frequency distributions
- Scraped the web to obtain the lyrics of three Italian singer-songwriters, in order to create one corpus per songwriter
- Created a frequency distribution to see which words are most frequent in their text production

STEP 2 - Spearman correlation 
- Compared the rankings to see the correlation between the corpora
- Removed stopwords from the corpora
- Recalculated the correlation

- Found common words among the three singers
- Recalculated the correlation between common word frequencies (with and without stopwords)


STEP 3 - Weirdness Index
- Calculated the weirdness index by taking into account the obtained corpora and a more generic corpus from Wikipedia

**Authors considered in this project:**

- Fabrizio De André (Genoa, 1940)
- Giorgio Gaber (Milan, 1939)
- Francesco Guccini (Modena, 1940)

These three singer-songwriters were born in the same period, although in different cities of Northern Italy.

In [1]:
# Define a function to clean the text scraped from the webpage by using regular expressions

import re

def clean_text(dirty_text):
    
    clean = re.sub(r"(\n)", " ", dirty_text)    # spaces instead of \n
    clean = re.sub(r"(\xa0)|(\ufeff)|(1ª Parte:)|(2ª Parte:)|(…)", "", clean)        # clean from other elements
    clean = re.sub(r"(Il video presente)+.*(puramente divulgativo)", "", clean)
    clean = re.sub(r"’", "' ", clean)           # replace weird apostrophe
    clean = re.sub(r"‘", "'", clean)
    clean = re.sub(r"(Don Chisciotte:)|(Sancho Panza:)|( Don Chisciotte e Sancho Panza insieme:)|(F:)|(A:)", "", clean) 
    clean = re.sub(r"(Canzone popolare francese del XIV secolo tradotta da De  André)", "", clean)
    clean = re.sub(r"\(Due invocazioni e un atto d' accusa\)", "", clean)
    clean = re.sub(r"\(Leggenda del Re infelice\)", "", clean)
    clean = re.sub(r"(1ª PARTE)|(2ª PARTE)|\(1° cliente\)|\(2° cliente\)|\(3° cliente\)|\(4° cliente\)", "", clean)
    clean = re.sub(r"-strumentale-", "", clean)
    clean = re.sub(r"(bela)", "bella", clean)     # clean typos
    clean = re.sub(r"(ulitmo)", "ultimo", clean)
    clean = re.sub(r"( ra )", " fra", clean)
    clean = re.sub(r"l\?", "l' ", clean)
    clean = re.sub(r"u'", "ù", clean)
    clean = re.sub(r"i'", "ì", clean)
    clean = re.sub(r"a'", "à", clean)
    clean = re.sub(r"(E')", "è", clean)
    clean = re.sub(r"( e' )", " è ", clean)
    clean = re.sub(r"(perchè)", "perché", clean)
    clean = re.sub(r"“|”|«|»", "", clean)       # clean from quotation marks
    
    return clean

In [2]:
# Get the text as a string from the "Testi e Traduzioni" website using requests, then clean it

import requests 
from bs4 import BeautifulSoup

def import_text(path):
      
    response = requests.get(path)         # get the text from the website
    response.encoding = "utf-8-sig"
    text = response.text
    
    soup = BeautifulSoup(text, "lxml")   # parse the text with BeautifulSoup
    
    l = []
    for tag in soup.find_all("p"):      # find_all does not have the get_text() method 
        l.append(tag.get_text())        # tag is a tag item: it has the get_text method
        
    da_pulire = "\n".join(l)              # turn the list into a string
    cleaned_text = clean_text(da_pulire)   # clean text by using the "clean_text" function
    
    return cleaned_text

## Francesco Guccini

In [3]:
# Songs in dialect that should not be taken into consideration

canzoni_dialetto_guccini = ["Vai al testo e traduzione di Al trist", "Vai al testo e traduzione di La Ziatta (La Tieta)"]

In [53]:
# Function to extract the links to the song pages from the main webpage of "Testi e Traduzioni"
# We want to skip songs in dialect

import requests 
from bs4 import BeautifulSoup

def get_links_to_lyrics_guccini(path):
    response1 = requests.get(path)  # main webpage
    response1.encoding = "utf-8-sig"
    text1 = response1.text
    
    soup1 = BeautifulSoup(text1, "lxml")
    
    li = soup1.find("div", class_= "center").find_all("li")   # extract the urls from the text body
   
    lista_link = []
    
    for element in li:
        links = element.find_all("a")
        
        for link in links:
            if link["title"] in canzoni_dialetto_guccini:   # skip links to songs in dialect
                pass
            else:
                lista_link.append(link["href"])
    return lista_link

In [54]:
lista_url_pag1 = get_links_to_lyrics_guccini("http://www.testietraduzioni.com/cantanti/f/francesco-guccini")
lista_url_pag1

['http://www.testietraduzioni.com/cantanti/f/francesco-guccini/milia.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/100-pennsylvania-ave.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/acque.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/addio.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/amerigo.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/antenr.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/argentina.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/asia.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/auschwitz.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/autogrill.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/autunno.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/ballando-con-una-sconosciuta.html',
 'http://www.testietraduzioni.com/

In [2]:
# Import webdriver

from selenium import webdriver

driver = webdriver.Chrome(executable_path= "/Users/LNV/Desktop/Jupiter_Notebook/executables/chromedriver.exe")

In [146]:
# Get urls from following pages
# The function clicks on pages 2-3 to extract the songs' urls

def click_on_page_get_links_guccini(path, page_number_as_string):  
    driver.get(path)                                                # give driver the path to the main webpage
    link = driver.find_element_by_link_text(page_number_as_string)  # webdriver finds the link to page 2 or 3 in the main page
    link.click()                                                    # click on the number
    current_url = driver.current_url                                # get the url of the page I have clicked on
    
    url_list = get_links_to_lyrics_guccini(current_url)      # the previously defined function extracts the urls to the songs' pages
    return url_list

In [148]:
lista_url_pag2 = click_on_page_get_links_guccini("http://www.testietraduzioni.com/cantanti/f/francesco-guccini", "2")
lista_url_pag2

['http://www.testietraduzioni.com/cantanti/f/francesco-guccini/gli-artisti.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/gulliver.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/ho-ancora-la-forza.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/i-fichi.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-3-dicembre-del-39.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-bello.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-caduto.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-compleanno.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-frate.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-matto.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-pensionato.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/il-sociale-e-lantisociale.html',


In [9]:
lista_url_pag3 = click_on_page_get_links_guccini("http://www.testietraduzioni.com/cantanti/f/francesco-guccini", "3")
lista_url_pag3

['http://www.testietraduzioni.com/cantanti/f/francesco-guccini/notti.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/odysseus.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/ophelia.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/parole.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/per-quando-tardi.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/piazza-alimonda.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/piccola-citt.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/piccola-storia-ignobile.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/primavera-59.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/primavera-di-praga.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/quattro-stracci.html',
 'http://www.testietraduzioni.com/cantanti/f/francesco-guccini/quel-giorno-daprile

In [10]:
# Two of De André's songs are repeated twice - we want to avoid getting double lyrics

urls_to_skip = ["http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/oceano-2.html", "http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/verranno-a-chiederti-del-nostro-amore-2.html"]

In [11]:
# Get all the lyrics by using the lists of urls and the previously defined "import_text" function

def get_lyrics(lista_url):
    
    lista_testi = []
    
    for link in lista_url:
        if link not in urls_to_skip:     # skip double songs 
            testo = import_text(link)    # get the lyrics through the "import_text" function
            lista_testi.append(testo)    # append the lyrics to a list
    testi = "\n".join(lista_testi)      # turn the list into a string
    return testi

In [132]:
# By using the function defined above, extract the lyrics of the songs listed in the webpages

testi_1_guccini = get_lyrics(lista_url_pag1)
print(testi_1_guccini)

Le Alpi, si sa, sono un muro di sasso una diga confusa, fanno tabula rasa di noi che qui sotto, lontano, più in basso abbiamo la casa la casa ed i piedi in questa spianata di sole che strozza la gola alle rane di nebbia compatta, scabrosa, stirata che sembra di pane, ed una strada antica come l' uomo marcata ai bordi dalla fantasie di un duomo e fiumi, falsi avventurieri che trasformano i padani in marinai non veri. Emilia sdraiata fra i campi e sui prati lagune e piroghe delle terramare guerrieri del Nord dai capelli gessati ne hai visti passare! Emilia allungata fra l' olmo e il vigneto voltata a cercare quel mare mancante e il monte Appennino rivela il segreto e diventa un gigante. Lungo la strada tra una piazza e un duomo hai messo al mondo questa specie d' uomo vero, aperto, finto, strano chiuso, anarchico, verdiano. Brutta razza,l' emiliano! Emilia sognante fra l' oggi e il domani di cibo e motori, di lusso e balere Emilia di facce, di grida, di mani sarà un grande piacere vedere

In [149]:
testi_2_guccini = get_lyrics(lista_url_pag2)
print(testi_2_guccini)

Gli artisti non nascono artisti, non sembrano strani animali  ma nascono un po'  come tutti, come individui normali.   Hanno lacrime e riso, hanno due occhi e due mani,  hanno stampata sul viso l' impronta di esseri umani.   Poi, appena un po'  cresciuti, li avvolge una strana espressione  e appare sui volti convinti la stigmate della vocazione.   Non sperano di fare il pompiere, l' astronauta o il ciclista,  non vogliono un comune mestiere ma vogliono essere artista.  Non sono più alti o più belli ma indossano panni curiosi,  son quelli che lancian coltelli sognando di esser famosi.   C' è quello che annaspa e si pigia da abile contorsionista,  chiudendosi in una valigia con un costume d' artista.   E girano il mondo dei circhi, vagando di quà e di là,  paghi d' applausi sol quando si inchinano e gridan Voilà!   E amano donne fedeli, che aspettano nel carrozzone,  rattoppano una calzamaglia e adorano il loro campione.  Ci sono il cantante e l' attore, il poeta, lo stilista,  spesso so

In [134]:
testi_3_guccini = get_lyrics(lista_url_pag3)
print(testi_3_guccini)

Notti che durano non so quante ore cascate impetuose o gocce in un mare notti che bruciano su una ferita, notti boccate di vita  Notti indelebili che marchiano un volto notti invisibili senza raccolto notti da incorniciare, ore di plastica da riciclare. Notti che spaccano il calendario senza brindare per l' anniversario vasi di tempo che invecchiano l' uomo e le facciate di un duomo.  E con coraggio potrai viverle fino alla fine o chiuderle in una bacheca, ma è un' esistenza più cieca.  Con l' incoscienza potrai spenderle tutte in un sogno per annegare il rimpianto e dare voce al tuo tempo o forse le dimenticherai forse le ascolterai.  Notti in difesa giocate di sponda lì ad aspettare la tua giusta onda notti da preda, da belva o da insetto fuggite o prese di petto impermeabili ad ogni ricordo  C' è chi ne parla ma io resto sordo notti acquazzoni d' estete nubi gonfie di storie perdute  Le notti scivolano o raschiano il fondo lievi di schiuma o pugni di piombo, imprevedibili come naufr

In [135]:
# One important Guccini's song was missing from the list!

per_fare_un_uomo = import_text("http://www.testietraduzioni.com/cantanti/n/nomadi/per-fare-un-uomo.html")
print(per_fare_un_uomo)

E cade la pioggia  E cambia ogni cosa  La morte e la vita  Non cambiano mai  L' inverno è tornato  L' estate è finita  La morte e la vita  Rimangono uguali.  La morte e la vita  rimangono uguali.  Per fare un uomo  Ci vogliono vent' anni  Per fare un bimbo  Un' ora d' amore  Per una vita  Migliaia di ore  Per il dolore  È abbastanza un minuto.  Per il dolore  È abbastanza un minuto.  E verrà il tempo  Di dire parole  Quando la vita  Una vita darà  E verrà il tempo  Di fare l' amore  Quando l' inverno  Più a nord se ne andrà.  Poi andremo via  Come fanno gli uccelli  Che dove vanno  Nessuno lo sa  Ma verrà il tempo  E quel cielo vedremo  Quando l' inverno  Dal nord tornerà.  Quando l' inverno  Dal nord tornerà  E cade la pioggia  E cambia ogni cosa  La morte e la vita  Non cambiano mai  L' estate è passata  L' inverno è alle porte  La vita e la morte  Rimangono uguali.  La vita e la morte  Rimangono uguali 


In [150]:
# Create file with all Guccini's lyrics

corpus_testi_guccini = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_guccini.txt", "w", encoding="utf8")
corpus_testi_guccini.write(testi_1_guccini + "\n" + testi_2_guccini + "\n" + testi_3_guccini + "\n" + per_fare_un_uomo)
corpus_testi_guccini.close()

In [1]:
# Visualise the file content and store it in a variable

corpus_testi_guccini = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_guccini.txt", "r", encoding="utf8").read()
print(corpus_testi_guccini)

Le Alpi, si sa, sono un muro di sasso una diga confusa, fanno tabula rasa di noi che qui sotto, lontano, più in basso abbiamo la casa la casa ed i piedi in questa spianata di sole che strozza la gola alle rane di nebbia compatta, scabrosa, stirata che sembra di pane, ed una strada antica come l' uomo marcata ai bordi dalla fantasie di un duomo e fiumi, falsi avventurieri che trasformano i padani in marinai non veri. Emilia sdraiata fra i campi e sui prati lagune e piroghe delle terramare guerrieri del Nord dai capelli gessati ne hai visti passare! Emilia allungata fra l' olmo e il vigneto voltata a cercare quel mare mancante e il monte Appennino rivela il segreto e diventa un gigante. Lungo la strada tra una piazza e un duomo hai messo al mondo questa specie d' uomo vero, aperto, finto, strano chiuso, anarchico, verdiano. Brutta razza,l' emiliano! Emilia sognante fra l' oggi e il domani di cibo e motori, di lusso e balere Emilia di facce, di grida, di mani sarà un grande piacere vedere

## Fabrizio De André

In [18]:
# Songs in dialect that should not be taken into consideration

canzoni_dialetto_deandre = ["Vai al testo e traduzione di Â cúmba", "Vai al testo e traduzione di Â duménega", "Vai al testo e traduzione di Â pittima", "Vai al testo e traduzione di A Cimma", "Vai al testo e traduzione di Ave Maria (sarda)", "Vai al testo e traduzione di Crêuza de mä", "Vai al testo e traduzione di D’ä mê riva", "Vai al testo e traduzione di Jamin-a", "Vai al testo e traduzione di Laudate Dominum", "Vai al testo e traduzione di Mégu megún", "Vai al testo e traduzione di Monti di mola", "Vai al testo e traduzione di Sidùn", "Vai al testo e traduzione di Sinàn Capudàn Pascià", "Vai al testo e traduzione di Zirichiltaggia"]

In [19]:
# Modify the function in order to skip songs in dialect

def get_links_to_lyrics_deandre(path):
    response1 = requests.get(path)  # main webpage
    response1.encoding = "utf-8-sig"
    text1 = response1.text
    
    soup1 = BeautifulSoup(text1, "lxml")
    
    li = soup1.find("div", class_= "center").find_all("li")   # extract the urls from the text body
   
    lista_link = []
    
    for element in li:
        links = element.find_all("a")
        
        for link in links:
            if link["title"] not in canzoni_dialetto_deandre:    # skip links to songs in dialect
                lista_link.append(link["href"])
    return lista_link

In [20]:
lista_url_pag1_deandre = get_links_to_lyrics_deandre("http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre")
lista_url_pag1_deandre

['http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/al-ballo-mascherato.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/amico-fragile.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/amore-che-vieni-amore-che-vai.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/andrea.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/anime-salve.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/ave-maria.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/avventura-a-durango.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/ballata-degli-impiccati.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/ballata-dellamore-cieco.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/barbara.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/bocca-di-rosa.html',
 'http://www.testietraduzioni.com/cantanti/f/fa

In [21]:
# Get urls from following pages
# The function clicks on pages 2-3 to extract the songs' urls

def click_on_page_get_links_deandre(path, page_number_as_string):  
    driver.get(path)                                                # give driver the path to the main webpage
    link = driver.find_element_by_link_text(page_number_as_string)  # webdriver finds the link to page 2 or 3 in the main page
    link.click()                                                    # click on the number
    current_url = driver.current_url                                # get the url of the page I have clicked on
    
    url_list = get_links_to_lyrics_deandre(current_url)      # the previously defined function extracts the urls to the songs' pages
    return url_list

In [22]:
# Use function defined above to click on following pages and get the urls

lista_url_pag2_deandre = click_on_page_get_links_deandre("http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre", "2")
lista_url_pag2_deandre

['http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-re-fa-rullare-i-tamburi.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-ritorno-di-giuseppe.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-sogno-di-maria.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-suonatore-jones.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-testamento.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/il-testamento-di-tito.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/introduzione.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/inverno-2.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/khorakhan-a-forza-di-essere-vento.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/linfanzia-di-maria.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/la-ballata-del-mich.html',
 'http://ww

In [23]:
lista_url_pag3_deandre = click_on_page_get_links_deandre("http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre", "3")
lista_url_pag3_deandre

['http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/recitativo.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/rimini.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/si-fosse-foco.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/sally.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/se-ti-tagliassero-a-pezzetti.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/secondo-intermezzo.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/si-chiamava-ges.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/smisurata-preghiera.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/sogno-numero-due.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/spiritual.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/suzanne.html',
 'http://www.testietraduzioni.com/cantanti/f/fabrizio-de-andre/tema-di-ri

In [24]:
# Extract lyrics of songs from page 1

testi_1_deandre = get_lyrics(lista_url_pag1_deandre)
print(testi_1_deandre)

Cristo drogato da troppe sconfitte cede alla complicità di Nobel che gli espone la praticità di un' eventuale premio della bontà.  Maria ignorata da un Edipo ormai scaltro mima una sua nostalgia di natività, io con la mia bomba porto la novità, la bomba che debutta in società, al ballo mascherato della celebrità.  Dante alla porta di Paolo e Francesca spia chi fa meglio di lui: lì dietro si racconta un amore normale ma lui saprà poi renderlo tanto geniale.  E il viaggio all' inferno ora fallo da solo con l' ultima invidia lasciata là sotto un lenzuolo, sorpresa sulla porta d' una felicità la bomba ha risparmiato la normalità, al ballo mascherato della celebrità.  La bomba non ha una natura gentile ma spinta da imparzialità sconvolge l' improbabile intimità di un' apparente statua della Pietà.  Grimilde di Manhattan, statua della libertà, adesso non ha più rivali la tua vanità e il gioco dello specchio non si ripeterà Sono più bella io o la statua della Pietà  dopo il ballo mascherato d

In [25]:
testi_2_deandre = get_lyrics(lista_url_pag2_deandre)
print(testi_2_deandre)

 Il re fa rullare i tamburi Il re fa rullare i tamburi  vuol sceglier fra le dame un nuovo e fresco amore ed è la prima che ha veduto che gli ha rapito il cuore Marchese la conosci tu? marchese la conosci tu chi è quella graziosa? Ed il marchese disse al re Maestà è la mia sposa Tu sei più felice di me, tu sei più felice di me d' aver dama si bella signora si compita se tu vorrai cederla a me sarà la favorita Signore se non foste il re, signore se non foste il re v' intimerei prudenza ma siete il sire, siete il re vi devo l' obbedienza  marchese vedrai passerà, marchese vedrai passerà d' amor la sofferenza io ti farò nelle mie armate maresciallo di Francia  Addio per sempre mia gioia addio per sempre mia bella  addio dolce amore devi lasciarmi per il re ed io ti lascio il cuore La regina ha raccolto dei fiori la regina ha raccolto dei fiori  celando la sua offesa ed il profumo di quei fiori uccise la marchesa. 
Stelle, già dal tramonto, si contendono il cielo a frotte, luci meticolose 

In [26]:
testi_3_deandre = get_lyrics(lista_url_pag3_deandre)
print(testi_3_deandre)

  Uomini senza fallo, semidei che vivete in castelli inargentati che di gloria toccaste gli apogei noi che invochiam pietà siamo i drogati Dell' inumano varcando il confine conoscemmo anzitempo la carogna che ad ogni ambito sogno mette fine: che la pietà non vi sia di vergogna Banchieri, pizzicagnoli, notai coi ventri obesi e le mani sudate coi cuori a forma di salvadanai noi che invochiam pietà fummo traviate Navigammo su fragili vascelli per affrontar del mondo la burrasca ed avevamo gli occhi troppo belli: che la pietà non vi rimanga in tasca Giudici eletti, uomini di legge noi che danziam nei vostri sogni ancora siamo l' umano desolato gregge di chi morì con il nodo alla gola Quanti innocenti all' orrenda agonia votaste decidendone la sorte e quanto giusta pensate che sia una sentenza che decreta morte ? Uomini cui pietà non convien sempre mal accettando il destino comune, andate, nelle sere di novembre, a spiar delle stelle al fioco lume, la morte e il vento, in mezzo ai camposant

In [27]:
# Create file with all De André's lyrics

corpus_testi_de_andre = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_de_andre.txt", "w", encoding="utf8")
corpus_testi_de_andre.write(testi_1_deandre + "\n" + testi_2_deandre + "\n" + testi_3_deandre)
corpus_testi_de_andre.close()

In [2]:
# Visualise the file content and store it in a variable

corpus_testi_de_andre = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_de_andre.txt", "r", encoding="utf8").read()
print(corpus_testi_de_andre)

Cristo drogato da troppe sconfitte cede alla complicità di Nobel che gli espone la praticità di un' eventuale premio della bontà.  Maria ignorata da un Edipo ormai scaltro mima una sua nostalgia di natività, io con la mia bomba porto la novità, la bomba che debutta in società, al ballo mascherato della celebrità.  Dante alla porta di Paolo e Francesca spia chi fa meglio di lui: lì dietro si racconta un amore normale ma lui saprà poi renderlo tanto geniale.  E il viaggio all' inferno ora fallo da solo con l' ultima invidia lasciata là sotto un lenzuolo, sorpresa sulla porta d' una felicità la bomba ha risparmiato la normalità, al ballo mascherato della celebrità.  La bomba non ha una natura gentile ma spinta da imparzialità sconvolge l' improbabile intimità di un' apparente statua della Pietà.  Grimilde di Manhattan, statua della libertà, adesso non ha più rivali la tua vanità e il gioco dello specchio non si ripeterà Sono più bella io o la statua della Pietà  dopo il ballo mascherato d

## Giorgio Gaber

In [29]:
# The website used for the other two authors had only a few of Gaber's songs, so I scraped them from a different website
# Since it is a different website, it has a different html, hence it needs a different function to extract the links 
# to the song pages from the main webpage

def get_links_to_lyrics_gaber(path):
    response1 = requests.get(path)  # main webpage
    response1.encoding = "utf-8-sig"
    text1 = response1.text
    
    soup1 = BeautifulSoup(text1, "lxml")
    
    li = soup1.find("ul", class_= "liststampa").find_all("li")   # extract the urls from the text body
   
    lista_link = []
    
    for element in li:
        links = element.find_all("a")
        
        for link in links:
            lista_link.append(link["href"])
    return lista_link

In [30]:
lista_url_gaber = get_links_to_lyrics_gaber("http://www.giorgiogaber.org/index.php?page=testi")
lista_url_gaber

['http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=105',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=158',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=246',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=247',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=187',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=106',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=104',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=194',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=107',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=209',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=262',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=136',
 'http://www.giorgiogaber.org/index.php?page=testi-veditesto&codTesto=115',
 'http://www

In [31]:
# Since it is a different website, it needs a different "cleaning procedure"

def clean_text_2(dirty_text):
    
    clean = re.sub(r"(\.\.\.)", "", dirty_text)
    clean = re.sub(r"a\)|b\)", "", clean)
    clean = re.sub(r"\[Voce fuori campo:]|\[G:]|\[parlato]|\[parlato:]|\(Parlato\)", "", clean)
    clean = re.sub(r"(\.\.)", "", clean)
    clean = re.sub(r"’|‘", "'", clean)          
    clean = re.sub(r"“|”", '"', clean)
    clean = re.sub(r"\(1\)", "", clean)
    clean = re.sub(r"l\'", "l' ", clean)
    clean = re.sub(r"l'|L'", "l' ", clean)
    clean = re.sub(r"d'|D'", "d' ", clean)
    clean = re.sub(r"un'|Un'", "un' ", clean)
    clean = re.sub(r"va\.", "va ", clean)
    clean = re.sub(r"c'|C'", "c' ", clean)
    clean = re.sub(r"(\…)", "", clean)
    clean = re.sub(r"dài", "dai", clean)
    
    return clean

In [32]:
# The new cleaning function replaces the previous one in the function that returns the cleaned text from the relevant website

def import_text_2(path):
      
    response = requests.get(path)         # get the text from the website
    response.encoding = "utf-8-sig"
    text = response.text
    
    soup = BeautifulSoup(text, "lxml")     # parse the text with BeautifulSoup
    
    da_pulire = soup.find("p", class_= "testo").get_text()
   
    cleaned_text = clean_text_2(da_pulire)   # clean text by using the "clean_text" function
    
    return cleaned_text

In [33]:
# Get all the lyrics by using the lists of urls and the previously defined "import_text" function

def get_lyrics_2(lista_url):
    
    lista_testi = []
    
    for link in lista_url:
        testo = import_text_2(link)        # get the lyrics through the "import_text" function
        lista_testi.append(testo)          # append the lyrics to a list
    testi = "\n".join(lista_testi)             # turn the list into a string
    return testi

In [34]:
# By using the function defined above, extract the lyrics of the songs listed in the webpage

testi_gaber = get_lyrics_2(lista_url_gaber)
print(testi_gaber)

Ma la Storia lasciò l'  uomo
al numero 1981
e l'  uomo come congelato
non intravedeva il suo destino.
Non era il capolinea
qualcosa doveva accadere
lo suggeriva una fede spontanea
che non era ancora il tempo di morire.

Il vecchio saggio
e il bimbo appena nato
guardavano la notte
dove il caso è in agguato.

E la notte
lasciava intravedere la notte
col trucco metafisico e scioccante
che l'  è proprio
le cose che riuscivano a stupire
il bimbo e il vecchio.
Come ad esempio su di un cielo eterno
un grattacielo illuminato di pistacchio.

Il vecchio saggio
e il bimbo tra le braccia della mamma
di fronte a quella strana meraviglia
rinnovarono il dilemma
se quelle cose colorate e straordinarie
sarebbero col tempo diventate
se a Dio fosse piaciuto
necessarie.

Ma di una cosa siamo certi
che i loro occhi vedevano
non so se con fiducia o senza scampo
quell'  enorme assurdità che è il tempo.

Signore Iddio, non so se faccia bene o faccia male
assistere ogni tanto al tuo definitivo e ricorrente fun

In [35]:
# Create file with all Gaber's lyrics

corpus_testi_gaber = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_gaber.txt", "w", encoding="utf8")
corpus_testi_gaber.write(testi_gaber)
corpus_testi_gaber.close()

In [3]:
# Visualise the file content and store it in a variable

corpus_testi_gaber = open("/Users/LNV/Desktop/Jupiter_Notebook/project/corpus_testi_gaber.txt", "r", encoding="utf8").read()
print(corpus_testi_gaber)

Ma la Storia lasciò l'  uomo

al numero 1981

e l'  uomo come congelato

non intravedeva il suo destino.

Non era il capolinea

qualcosa doveva accadere

lo suggeriva una fede spontanea

che non era ancora il tempo di morire.


Il vecchio saggio

e il bimbo appena nato

guardavano la notte

dove il caso è in agguato.


E la notte

lasciava intravedere la notte

col trucco metafisico e scioccante

che l'  è proprio

le cose che riuscivano a stupire

il bimbo e il vecchio.

Come ad esempio su di un cielo eterno

un grattacielo illuminato di pistacchio.


Il vecchio saggio

e il bimbo tra le braccia della mamma

di fronte a quella strana meraviglia

rinnovarono il dilemma

se quelle cose colorate e straordinarie

sarebbero col tempo diventate

se a Dio fosse piaciuto

necessarie.


Ma di una cosa siamo certi

che i loro occhi vedevano

non so se con fiducia o senza scampo

quell'  enorme assurdità che è il tempo.


Signore Iddio, non so se faccia bene o faccia male

assistere ogni tanto a

## Word frequency comparison

In [4]:
# Define a function to return the word frequency of a given text

import nltk
import string

def count_frequency(text_from_file):
    
    list_of_tokens = nltk.word_tokenize(text_from_file.lower())   # tokenize the text
    
    # problem: the apostrophe is treated as a token, thus words like "po'" and "l'" get separated
    # use the package Multi-Word Expression Tokenizer to rejoin them

    no_tokenizer = nltk.tokenize.MWETokenizer(separator="") # separator to have no space between the two elements 
    no_tokenizer.add_mwe(("l", "'"))                        # (it would be _ by default)
    no_tokenizer.add_mwe(("po", "'"))                       # list elements that have to be kept together
    no_tokenizer.add_mwe(("d", "'"))
    no_tokenizer.add_mwe(("c", "'"))
    no_tokenizer.add_mwe(("s", "'"))
    no_tokenizer.add_mwe(("t", "'"))
    no_tokenizer.add_mwe(("n", "'"))
    no_tokenizer.add_mwe(("all", "'"))
    no_tokenizer.add_mwe(("cos", "'"))
    no_tokenizer.add_mwe(("com", "'"))
    no_tokenizer.add_mwe(("dov", "'"))
    no_tokenizer.add_mwe(("ch", "'"))
    no_tokenizer.add_mwe(("vent", "'"))
    no_tokenizer.add_mwe(("quell", "'"))
    no_tokenizer.add_mwe(("nell", "'")) 
    no_tokenizer.add_mwe(("dell", "'")) 
    
    tokens_with_apostrophe = no_tokenizer.tokenize(list_of_tokens)   # apply MWETokenizer to the tokenized text
    
    puncts = string.punctuation        # store the punctuation set in a variable
    
    d = {}
    for i in tokens_with_apostrophe:
        if i in puncts:         # remove punctuation 
            pass
        elif i == '``' or i == "''" or i == "--" or i=="–":   # remove weird quotes that keep appearing in Gaber's lyrics
            pass
        else:
            if i not in d:    # populate dictionary
                d[i] = 1
            else:
                d[i] += 1
    sorted_list = sorted(d.items(), key = lambda x : x[1], reverse = True)   # turn dictionary into sorted list
    return sorted_list 

In [5]:
# Word frequency in Guccini's songs

lista_frequenza_guccini = count_frequency(corpus_testi_guccini)
lista_frequenza_guccini

[('e', 1815),
 ('che', 1179),
 ('di', 1139),
 ('un', 980),
 ('la', 864),
 ('il', 845),
 ('non', 801),
 ('è', 722),
 ('in', 586),
 ('a', 586),
 ('ma', 480),
 ("l'", 431),
 ('per', 408),
 ('le', 397),
 ('i', 381),
 ('si', 346),
 ('o', 303),
 ('come', 301),
 ('una', 293),
 ('da', 262),
 ('con', 255),
 ('se', 250),
 ('io', 221),
 ('mi', 221),
 ("d'", 190),
 ('ti', 188),
 ('più', 186),
 ('del', 176),
 ('poi', 165),
 ('sono', 164),
 ('al', 162),
 ('sempre', 158),
 ('lo', 157),
 ('ci', 153),
 ('gli', 151),
 ('ed', 144),
 ('ha', 143),
 ("c'", 137),
 ('giorno', 137),
 ('quando', 136),
 ('solo', 134),
 ('ho', 129),
 ('era', 128),
 ('son', 127),
 ('mai', 127),
 ('perché', 126),
 ('ogni', 125),
 ('nel', 123),
 ('cosa', 121),
 ('ancora', 119),
 ('vita', 119),
 ('quel', 116),
 ('della', 114),
 ('già', 112),
 ('chi', 111),
 ('tempo', 109),
 ('tutto', 107),
 ('fra', 104),
 ('senza', 102),
 ('dei', 101),
 ('dove', 96),
 ('me', 93),
 ('mondo', 91),
 ('tu', 87),
 ('anche', 85),
 ('questo', 85),
 ('tutti'

**Most frequent nouns in Guccini's songs:**  
\
*giorno
\
vita
\
tempo*

In [6]:
# Word frequency in De André's songs
# The lower numbers can be explained by the fact that De André has more songs in dialect compared to the other authors, so 
# some songs were not considered

lista_frequenza_de_andre = count_frequency(corpus_testi_de_andre)
lista_frequenza_de_andre

[('e', 851),
 ('di', 754),
 ('il', 673),
 ('che', 654),
 ('la', 624),
 ('un', 565),
 ('non', 494),
 ('a', 427),
 ('per', 343),
 ('è', 337),
 ('le', 304),
 ('in', 281),
 ("l'", 264),
 ('una', 235),
 ('i', 212),
 ('si', 208),
 ('ma', 182),
 ('con', 174),
 ('al', 167),
 ('più', 156),
 ('da', 155),
 ('del', 154),
 ("d'", 153),
 ('mi', 148),
 ('amore', 143),
 ('ti', 142),
 ('gli', 140),
 ('se', 139),
 ('come', 137),
 ('lo', 136),
 ('ha', 128),
 ('senza', 108),
 ('nel', 102),
 ('chi', 99),
 ('ho', 93),
 ('alla', 87),
 ('della', 86),
 ('occhi', 86),
 ('suo', 84),
 ('tu', 84),
 ('io', 82),
 ('ci', 80),
 ('sono', 79),
 ('o', 79),
 ('quando', 78),
 ('dio', 76),
 ("c'", 75),
 ('ogni', 74),
 ('mio', 72),
 ('ora', 69),
 ('era', 65),
 ('perché', 65),
 ('mia', 64),
 ('mai', 64),
 ('cuore', 63),
 ('lei', 62),
 ('ancora', 60),
 ('sua', 59),
 ('sulla', 59),
 ('sul', 57),
 ('tuo', 57),
 ('cielo', 55),
 ('lui', 53),
 ('notte', 53),
 ('ed', 52),
 ('dove', 50),
 ('fu', 49),
 ('dal', 48),
 ('ad', 48),
 ('tem

**Most frequent nouns in De André's songs:** 
\
*amore
\
occhi
\
dio*

In [7]:
# Word frequency in Gaber's songs
# Many of Gaber's songs are very long and more prose-like compared to the other authors, which explains the high numbers 
# in word frequency

lista_frequenza_gaber = count_frequency(corpus_testi_gaber)
lista_frequenza_gaber

[('che', 3667),
 ('è', 3426),
 ('di', 3409),
 ('e', 3385),
 ('un', 3209),
 ('la', 2925),
 ('non', 2907),
 ('il', 2196),
 ('si', 1545),
 ('a', 1494),
 ('mi', 1445),
 ('una', 1310),
 ("l'", 1292),
 ('per', 1187),
 ('in', 1183),
 ('ma', 1104),
 ('più', 1034),
 ('le', 986),
 ('come', 962),
 ('sono', 955),
 ('io', 934),
 ('con', 863),
 ('i', 847),
 ('se', 795),
 ('da', 704),
 ("c'", 651),
 ('ci', 630),
 ('anche', 627),
 ('gli', 579),
 ('lo', 577),
 ('ha', 566),
 ('fa', 546),
 ('ho', 532),
 ('tutto', 490),
 ('del', 488),
 ("po'", 483),
 ('poi', 445),
 ('sì', 445),
 ('ti', 435),
 ('no', 429),
 ('perché', 422),
 ('sempre', 417),
 ('così', 396),
 ('al', 390),
 ('male', 388),
 ('uomo', 377),
 ('tutti', 364),
 ('cosa', 358),
 ('me', 349),
 ('della', 325),
 ('bene', 325),
 ('quando', 321),
 ('o', 315),
 ('niente', 313),
 ('può', 312),
 ('mai', 301),
 ('noi', 299),
 ('uno', 297),
 ('era', 295),
 ('mondo', 290),
 ('senza', 284),
 ('mia', 273),
 ('lui', 259),
 ('solo', 256),
 ('va', 255),
 ('mio', 25

**Most frequent nouns in Gaber's songs:**
\
\
*uomo
\
mondo
\
amore*

In [78]:
# Let us compare the frequency of certain words in the authors' lyrics

# "Amore" is one of the most frequent nouns in De André's songs

for el1, el2 in lista_frequenza_de_andre:
    if el1 == "amore" or el1 == "amor":
        print(el1, el2)

amore 143
amor 10


In [79]:
# It is very frequent also in Gaber's songs

for el1, el2 in lista_frequenza_gaber:
    if el1 == "amore" or el1 == "amor":
        print(el1, el2)

amore 249
amor 10


In [153]:
# Less frequent in Guccini's songs

for el1, el2 in lista_frequenza_guccini:
    if el1 == "amore" or el1 == "amor":
        print(el1, el2)

amore 49
amor 3


In [81]:
# The word "Dio" appears most frequently in Gaber's songs

for el1, el2 in lista_frequenza_gaber:
    if el1 == "dio":
        print(el1, el2)

dio 142


In [142]:
for el1, el2 in lista_frequenza_de_andre:
    if el1 == "dio":
        print(el1, el2)

dio 76


In [168]:
for el1, el2 in lista_frequenza_guccini:
    if el1 == "dio":
        print(el1, el2)

dio 36


In [169]:
# The word "wine" is repeated as many times as the word "Dio" in Guccini's songs!

for el1, el2 in lista_frequenza_guccini:
    if el1 == "vino":
        print(el1, el2)

vino 36


In [85]:
# In De André's songs, "wine" appears only 7 times

for el1, el2 in lista_frequenza_de_andre:
    if el1 == "vino":
        print(el1, el2)

vino 7


In [86]:
# In Gaber, only 6 times

for el1, el2 in lista_frequenza_gaber:
    if el1 == "vino":
        print(el1, el2)

vino 6
